<a href="https://colab.research.google.com/github/denklewer/ray-custom-agents/blob/master/trpo_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!sudo apt-get install -y build-essential curl unzip psmisc
!pip install cython==0.29.0
!git clone https://github.com/ray-project/ray.git
!ray/ci/travis/install-bazel.sh
!pip install lz4
!pip install setproctitle
!mv ray ray-distr
!pip install -e ray-distr/python/. --verbose  # Add --user if you see a permission denied error.

In [0]:
import sys
import os
sys.path.append(os.path.join("", '/content/ray-distr/python')) # To find local version of the library

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import gym
import logging
import ray
from ray import tune
from ray.rllib.evaluation import PolicyGraph, PolicyEvaluator, SampleBatch
from ray.rllib.evaluation.metrics import collect_metrics
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy
import scipy.signal
from ray.rllib.utils.annotations import override
logger = logging.getLogger(__name__)
from ray.rllib.evaluation.postprocessing import compute_advantages,Postprocessing

from torch.autograd import Variable
parser = argparse.ArgumentParser()
parser.add_argument("--gpu", action="store_true")
parser.add_argument("--num-iters", type=int, default=20)
parser.add_argument("--num-workers", type=int, default=2)

class TRPOAgent(nn.Module):
    def __init__(self, state_shape, n_actions, hidden_size=32):
        '''
        Here you should define your model
        You should have LOG-PROBABILITIES as output because you will need it to compute loss
        We recommend that you start simple:
        use 1-2 hidden layers with 100-500 units and relu for the first try
        '''
        nn.Module.__init__(self)

        self.n_actions = n_actions
        self.state_hape = state_shape

        self.model = nn.Sequential(
            nn.Linear(state_shape[0], hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
            nn.LogSoftmax()
        )

    def forward(self, states):
        """
        takes agent's observation (Variable), returns log-probabilities (Variable)
        :param state_t: a batch of states, shape = [batch_size, state_shape]
        """

        # Use your network to compute log_probs for given state
        log_probs = self.model(states)
        return log_probs

    def get_log_probs(self, states):
        '''
        Log-probs for training
        '''

        return self.forward(states)

    def get_probs(self, states):
        '''
        Probs for interaction
        '''

        return torch.exp(self.forward(states))

    def act(self, obs, sample=True):
        '''
        Samples action from policy distribution (sample = True) or takes most likely action (sample = False)
        :param: obs - single observation vector
        :param sample: if True, samples from \pi, otherwise takes most likely action
        :returns: action (single integer) and probabilities for all actions
        '''

        probs = self.get_probs(Variable(torch.FloatTensor([obs]))).data.numpy()

        if sample:
            action = int(np.random.choice(self.n_actions, p=probs[0]))
        else:
            action = int(np.argmax(probs))

        return action, probs[0]



In [0]:
# UTIL FUNCTIONS
def get_cummulative_returns(r, gamma=1):
  """
  Computes cummulative discounted rewards given immediate rewards
        G_i = r_i + gamma*r_{i+1} + gamma^2*r_{i+2} + ...
        Also known as R(s,a)
  """
  r = np.array(r)
  assert r.ndim >= 1
  return scipy.signal.lfilter([1], [1, -gamma], r[::-1], axis=0)[::-1]

def conjugate_gradient(f_Ax, b, cg_iters=10, residual_tol=1e-10):
    """
    This method solves system of equation Ax=b using iterative method called conjugate gradients
    :f_Ax: function that returns Ax
    :b: targets for Ax
    :cg_iters: how many iterations this method should do
    :residual_tol: epsilon for stability
    """
    p = b.clone()
    r = b.clone()
    x = torch.zeros(b.size())
    rdotr = torch.sum(r * r)
    for i in range(cg_iters):
        z = f_Ax(p)
        v = rdotr / (torch.sum(p * z) + 1e-8)
        x += v * p
        r -= v * z
        newrdotr = torch.sum(r * r)
        mu = newrdotr / (rdotr + 1e-8)
        p = r + mu * p
        rdotr = newrdotr
        if rdotr < residual_tol:
            break
    return x


In [0]:
class CustomPolicy(PolicyGraph):
    """Example of a custom policy graph written from scratch.
    You might find it more convenient to extend TF/TorchPolicyGraph instead
    for a real policy.
    """

    def __init__(self, observation_space, action_space, config):
        PolicyGraph.__init__(self, observation_space, action_space, config)
        # example parameter
        self.w = 1.0
        self.observation_shape = observation_space.shape
        self.n_actions = action_space.n
        self.agent = TRPOAgent(self.observation_shape, self.n_actions)
        self.policy = []


    def compute_actions(self,
                        obs_batch,
                        state_batches,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        info_batch=None,
                        episodes=None,
                        **kwargs):
        # return random actions
        actions = []
        action_probs = []
        for obs in obs_batch:
            action, policy = self.agent.act(obs)
            actions.append(action)
            action_probs.append(policy)

        return actions, [], {"action_probs": action_probs}
      
    @override(PolicyGraph)
    def postprocess_trajectory(self,
                               sample_batch,
                               other_agent_batches=None,
                               episode=None):
        traj = {}
        for key in sample_batch:
            traj[key] = np.stack(sample_batch[key])
        traj["cummulative_returns"] = get_cummulative_returns(traj[SampleBatch.REWARDS])
        return SampleBatch(traj)


    def get_flat_params_from(self, model):
        params = []
        for param in model.parameters():
            params.append(param.data.view(-1))

        flat_params = torch.cat(params)
        return flat_params

    def set_flat_params_to(self, model, flat_params):
        prev_ind = 0
        for param in model.parameters():
            flat_size = int(np.prod(list(param.size())))
            param.data.copy_(
                flat_params[prev_ind:prev_ind + flat_size].view(param.size()))
            prev_ind += flat_size

    def get_loss(self, agent, observations, actions, cummulative_returns, old_probs):
        """
        Computes TRPO objective
        :param: observations - batch of observations
        :param: actions - batch of actions
        :param: cummulative_returns - batch of cummulative returns
        :param: old_probs - batch of probabilities computed by old network
        :returns: scalar value of the objective function
        """
        batch_size = observations.shape[0]
        log_probs_all = agent.get_log_probs(observations)
        probs_all = torch.exp(log_probs_all)

        probs_for_actions = probs_all[torch.arange(
            0, batch_size, out=torch.LongTensor()), actions]
        old_probs_for_actions = old_probs[torch.arange(
            0, batch_size, out=torch.LongTensor()), actions]

        # Compute surrogate loss, aka importance-sampled policy gradient
        Loss = -torch.mean(cummulative_returns * (probs_for_actions / old_probs_for_actions))

        return Loss

    def get_kl(self, agent, observations, actions, cummulative_returns, old_probs_all):
        """
        Computes KL-divergence between network policy and old policy
        :param: observations - batch of observations
        :param: actions - batch of actions
        :param: cummulative_returns - batch of cummulative returns (we don't need it actually)
        :param: old_probs - batch of probabilities computed by old network
        :returns: scalar value of the KL-divergence
        """
        batch_size = observations.shape[0]
        log_probs_all = agent.get_log_probs(observations)
        probs_all = torch.exp(log_probs_all)

        # Compute Kullback-Leibler divergence (see formula above)
        # Note: you need to sum KL and entropy over all actions, not just the ones agent took
        old_log_probs_all = torch.log(old_probs_all + 1e-10)

        kl = torch.sum(old_probs_all * (old_log_probs_all - log_probs_all)) / batch_size

        return kl

    def get_entropy(self, agent, observations):
        """
        Computes entropy of the network policy
        :param: observations - batch of observations
        :returns: scalar value of the entropy
        """

        observations = Variable(torch.FloatTensor(observations))

        batch_size = observations.shape[0]
        log_probs_all = agent.get_log_probs(observations)
        probs_all = torch.exp(log_probs_all)

        entropy = torch.sum(-probs_all * log_probs_all) / batch_size

        return entropy

    def linesearch(self, f, x, fullstep, max_kl):
        """
        Linesearch finds the best parameters of neural networks in the direction of fullstep contrainted by KL divergence.
        :param: f - function that returns loss, kl and arbitrary third component.
        :param: x - old parameters of neural network.
        :param: fullstep - direction in which we make search.
        :param: max_kl - constraint of KL divergence.
        :returns:
        """
        max_backtracks = 10
        loss, _, = f(x)
        for stepfrac in .5 ** np.arange(max_backtracks):
            xnew = x + stepfrac * fullstep
            new_loss, kl = f(xnew)
            actual_improve = new_loss - loss
            if kl.data.numpy() <= max_kl and actual_improve.data.numpy() < 0:
                x = xnew
                loss = new_loss
        return x

    def learn_on_batch(self, samples):
        # implement your learning code here
        max_kl = 0.01
        observations = samples['obs']
        actions = samples['actions']
        returns = samples['cummulative_returns']
        old_probs = samples['action_probs']
        loss, kl = self.update_step(observations, actions, returns, old_probs, max_kl)
        
        return {
            "loss": loss,
            "kl": kl
        }

    def get_weights(self):
        return self.get_flat_params_from(self.agent)

    def set_weights(self, weights):
        self.set_flat_params_to(self.agent, weights)
        
    def update_step(self, observations, actions, cummulative_returns, old_probs, max_kl):
      """
      This function does the TRPO update step
      :param: observations - batch of observations
      :param: actions - batch of actions
      :param: cummulative_returns - batch of cummulative returns
      :param: old_probs - batch of probabilities computed by old network
      :param: max_kl - controls how big KL divergence may be between old and new policy every step.
      :returns: KL between new and old policies and the value of the loss function.
      """
      agent = self.agent

      # Here we prepare the information
      observations = Variable(torch.FloatTensor(observations))
      actions = torch.LongTensor(actions)
      cummulative_returns = Variable(torch.FloatTensor(cummulative_returns))
      old_probs = Variable(torch.FloatTensor(old_probs))

      # Here we compute gradient of the loss function
      loss = self.get_loss(agent, observations, actions,
                      cummulative_returns, old_probs)
      grads = torch.autograd.grad(loss, agent.parameters())
      loss_grad = torch.cat([grad.view(-1) for grad in grads]).data

      def Fvp(v):
          # Here we compute Fx to do solve Fx = g using conjugate gradients
          # We actually do here a couple of tricks to compute it efficiently

          kl = self.get_kl(agent, observations, actions,
                      cummulative_returns, old_probs)

          grads = torch.autograd.grad(kl, agent.parameters(), create_graph=True)
          flat_grad_kl = torch.cat([grad.view(-1) for grad in grads])

          kl_v = (flat_grad_kl * Variable(v)).sum()
          grads = torch.autograd.grad(kl_v, agent.parameters())
          flat_grad_grad_kl = torch.cat(
              [grad.contiguous().view(-1) for grad in grads]).data

          return flat_grad_grad_kl + v * 0.1

      # Here we solveolve Fx = g system using conjugate gradients
      stepdir = conjugate_gradient(Fvp, -loss_grad, 10)

      # Here we compute the initial vector to do linear search
      shs = 0.5 * (stepdir * Fvp(stepdir)).sum(0, keepdim=True)

      lm = torch.sqrt(shs / max_kl)
      fullstep = stepdir / lm[0]


      # Here we get the start point
      prev_params = self.get_weights()

      def get_loss_kl(params):
          # Helper for linear search
          # Set new params and return loss + kl
          self.set_weights(params)
          return [self.get_loss(agent, observations, actions, cummulative_returns, old_probs),
                  self.get_kl(agent, observations, actions, cummulative_returns, old_probs)]

      # Here we find our new parameters
      new_params = self.linesearch(get_loss_kl, prev_params, fullstep, max_kl)

      return get_loss_kl(new_params)


In [0]:
from ray.tune.logger import pretty_print
from collections import OrderedDict

def training_workflow(config, reporter):
    # Setup policy and policy evaluation actors
    env = gym.make("CartPole-v0")
    policy = CustomPolicy(env.observation_space, env.action_space, {})
    workers = [
        PolicyEvaluator.as_remote().remote(lambda c: gym.make("CartPole-v0"),
                                           CustomPolicy)
        for _ in range(config["num_workers"])
    ]

    for it in range(config["num_iters"]):
        print("\n********** Iteration %i ************" % it)
        # Broadcast weights to the policy evaluation workers
        weights = ray.put({"default_policy": policy.get_weights()})
        for w in workers:
            w.set_weights.remote(weights)

        # Gather a batch of samples
        samples = SampleBatch.concat_samples(ray.get([w.sample.remote() for w in workers]))


        prev_t = 0
        paths = []
        counter = 0
        obervations, actions, rewards, action_probs, cum_returns = [], [], [], [], []
        for t in samples['t']:
          if t == 0 and prev_t != 0:
            
            path = {"observations": np.array(obervations),
                    "policy": np.array(action_probs),
                    "actions": np.array(actions),
                    "rewards": np.array(rewards),
                    "cumulative_returns": np.array(cum_returns),
                    }
            obervations, actions, rewards, action_probs = [], [], [], []
            paths.append(path)
          else:  
            obervations.append(samples['obs'][counter])
            actions.append(samples['actions'][counter])
            action_probs.append(samples['action_probs'][counter])
            rewards.append(samples['rewards'][counter])   
            cum_returns.append(samples['cummulative_returns'][counter])
            prev_t = t
          counter+=1

        # Improve the policy using the  batch


        loss_stats =   policy.learn_on_batch(samples)
        # Report current progress
        result =collect_metrics(remote_evaluators=workers)
#         print(pretty_print(result))
        
        
        
            # Report current progress
        episode_rewards = np.array([path["rewards"].sum() for path in paths])

        stats = OrderedDict()
        stats["Average sum of rewards per episode"] = episode_rewards.mean()
        stats["rewards"] = episode_rewards
        stats["Std of rewards per episode"] = episode_rewards.std()
        stats["KL between old and new distribution"] = loss_stats['kl'].data.numpy()
        stats["Surrogate loss"] = loss_stats['loss'].data.numpy()
        for k, v in stats.items():
            print(k + ": " + " " * (40 - len(k)) + str(v))
        reporter(**collect_metrics(remote_evaluators=workers))
        
        


In [155]:
ray.init()

2019-05-20 00:40:33,743	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.


Exception: ignored

In [163]:

tune.run(
        training_workflow,
        resources_per_trial={
            "gpu": 0,
            "cpu": 1,
            "extra_cpu": 1,
        },
        config={
            "num_workers": 1,
            "num_iters": 10,
        })

2019-05-20 00:42:45,544	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-05-20 00:42:45,545	INFO tune.py:223 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs
Memory usage on this node: 8.1/13.7 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 2/2 CPUs, 0/1 GPUs
Memory usage on this node: 8.1/13.7 GB
Result logdir: /root/ray_results/training_workflow
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - training_workflow_0:	RUNNING

(pid=16866) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=16866) Instructions for updating:
(pid=16866) non-resource variables are not supported in the long term
(pid=16872) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/compat.py:175: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=16872) Instructi

2019-05-20 00:42:50,488	INFO ray_trial_executor.py:180 -- Destroying actor for trial training_workflow_0. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


(pid=16866) Average sum of rewards per episode:       22.75
(pid=16866) rewards:                                  [ 6. 31. 23. 31.]
(pid=16866) Std of rewards per episode:               10.207228
(pid=16866) KL between old and new distribution:      0.009984713
(pid=16866) Surrogate loss:                           -14.456856
(pid=16866) 
(pid=16866) ********** Iteration 7 ************
(pid=16866) Average sum of rewards per episode:       39.0
(pid=16866) rewards:                                  [55. 23.]
(pid=16866) Std of rewards per episode:               16.0
(pid=16866) KL between old and new distribution:      0.009987923
(pid=16866) Surrogate loss:                           -21.609226
(pid=16866) 
(pid=16866) ********** Iteration 8 ************
(pid=16866) Average sum of rewards per episode:       33.5
(pid=16866) rewards:                                  [37. 30.]
(pid=16866) Std of rewards per episode:               3.5
(pid=16866) KL between old and new distribution:      0.0

[training_workflow_0]